In [2]:

%matplotlib inline
import requests
import re
import time
import os
import urllib.request
from typing import Union
import csv
import statistics
import pdf2docx

import pickle

import camelot
import typing
import io
import numpy as np
from docx import Document
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
%config Completer.use_jedi = False
from pathlib import Path
# import tqdm.notebook.tqdm as tqdm
from tqdm import tqdm_notebook
%matplotlib widget
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format=u'%(filename)+13s [ LINE:%(lineno)-4s] %(levelname)-8s %(message)s',
                    level=logging.DEBUG)

logger = logging.getLogger(__name__)

from docx2python import docx2python


In [25]:

class PdfParser:
    
    @staticmethod
    def convert_pdf_to_df(filename) -> list[pd.DataFrame]:
        tables = camelot.read_pdf(str(filename), line_tol=2, joint_tol=10, line_scale=40, copy_text=['v'], pages='1-end') # , flavor='stream' row_tol=10
        tables = [e.df for e in tables]
        return tables



# base = 'data_idus/pdf/converted/'
# file = '189273_2020_Rektor,_prorektory,_glavnyi_bukhgalter.pdf'
file = "D:/PROGR/LEARN_PYTHON/Declarator/declarations-parser/data_ids/pdf/converted/90569_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf"
# par = PdfParser()
# res = par.convert_pdf_to_df(file)
# res









In [3]:
folder = r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\docx'
folder_with_ok = r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\docx\cool\ok2'

ne_ok_ids = []
ok_ids = []
all_ids = []

for e in os.listdir(folder):
    if e.endswith('docx'):
        all_ids.append('_'.join(e.split('_')[:2]))

for e in os.listdir(folder_with_ok):
    if e.endswith('xlsx'):
        ok_ids.append('_'.join(e.split('_')[:2]))

# print(len(all_ids))
# print(len(ok_ids))



all_ids = set(all_ids)
ok_ids = set(ok_ids)

bad_ids = all_ids - ok_ids
bad_ids

#for e in os.listdir(folder_with_ok):
    


{'100206_2019',
 '101232_2018',
 '102031_2019',
 '102421_2019',
 '102541_2019',
 '102585_2019',
 '102586_2019',
 '102907_2019',
 '103026_2019',
 '104576_2019',
 '104619_2019',
 '176299_2020',
 '177336_2020',
 '177672_2020',
 '178290_2020',
 '178303_2020',
 '178403_2020',
 '178414_2020',
 '178423_2020',
 '179161_2020',
 '179252_2020',
 '179394_2020',
 '179467_2020',
 '179512_2020',
 '179590_2020',
 '179663_2020',
 '179911_2020',
 '180112_2020',
 '180258_2020',
 '180263_2020',
 '180371_2020',
 '180385_2020',
 '182639_2020',
 '183392_2020',
 '183413_2020',
 '184287_2018',
 '184503_2020',
 '184919_2020',
 '186491_2020',
 '186613_2020',
 '187331_2020',
 '187741_2020',
 '187742_2020',
 '188290_2020',
 '189123_2020',
 '189320_2020',
 '189322_2020',
 '189496_2020',
 '189701_2020',
 '189856_2020',
 '83292_2016',
 '83310_2016',
 '83311_2017',
 '83312_2018',
 '83316_2018',
 '83318_2017',
 '83321_2018',
 '83335_2018',
 '83336_2017',
 '83338_2018',
 '83339_2017',
 '83340_2018',
 '83341_2017',
 '833

In [2]:

class CorrectHeadersParser:

    '''класс для парсинга таблиц, у которых на месте колонки, которые нам нужны'''

    def table_splitter(self, table: pd.DataFrame) -> list[pd.DataFrame]:
        '''разделяет таблицы, в которых учреждение указано внутри таблицы'''

        def check_if_same(my_array: list) -> bool:
            
            '''проверяем одинаковые ли колонки'''

            if len(set(my_array))>1:
                return False
            return True

            # first = my_array[0]
            # for e in my_array[1:]:
            #     if e != first:
            #         return False
            # return True

        def get_indexes_to_split(table):
            index_to_split = []
            for e in range(len(table)):
                cols = table.iloc[e,:].values
                if check_if_same(cols):
                    index_to_split.append(e)
            return index_to_split


        def split_table(table: pd.DataFrame, index_to_split:Union[int, list[int]]) -> list[pd.DataFrame]:
            """разделяет таблицу в случае когда название учреждения поместили в середину вот так:

                -должность-  -имя-  -зарплата-
                        -ГБОУ школа 112-
                 директор     Ваня    100 руб

             """
            dfs = np.array_split(table, index_to_split)
            dfs = [e for e in dfs if len(e) > 0]

            result_dfs = []
            for df in dfs:
                office = df.iloc[0,:][0]
                df = df.iloc[1:,:] 
                df['office'] = office
                result_dfs.append(df)
            
            result_dfs = [e for e in result_dfs if not e.empty]
            try:
                result_dfs = pd.concat(result_dfs)
                return result_dfs
            except Exception as ex:
                print(ex)
                print('rogue file---', table)
                
        index_to_split = get_indexes_to_split(table)

        if not index_to_split:
            return table

        splitted_dfs = split_table(table, index_to_split)
        return splitted_dfs

        
    def concat_name(self, df: pd.DataFrame) -> pd.DataFrame:
        '''соединяем колонки ФИО, если они в разных'''
        
        if 'name' not in df.columns:
            return df
        
        names_df = df['name']
    
        if isinstance(names_df, str) or isinstance(names_df, pd.Series):
            return df  
    
        # TODO:
        # дропнуть маленькую колонку


        names = [' '.join(e) for e in names_df.values]     
        
        df.drop(columns=['name'], inplace=True)
        df['name'] = names
        return df


    def parse(self, table: pd.DataFrame) -> pd.DataFrame:
        # table = self.concat_name(table)
        table = self.table_splitter(table)
        return table



In [3]:


class DataCleaner:
    """убирает лишние данные"""

    @staticmethod
    def remove_unwanted_symbols(df):        
        # TODO: чистка всех колонок
        df = df.applymap(lambda x: str(x).replace('\n', ' '))
        return df
    
    @staticmethod
    def remove_unwanted_cells(df):
        # убирает ячейки с нумерацией
        # print('--- DataCleaner.remove_unwanted_cells ---', df.columns)
        #TODO: почему тут только должность?
        df = df[~df['position'].astype(str).str.isdigit()]
        return df
    
    @staticmethod
    def remove_short_rows(df):
        # удаляет ряды с недостаточными данными
        # ! должно применяться после выбора норм колонок
        to_remove = []
        for tup in df.itertuples():
            res = [len(str(e)) for e in tup]
            if statistics.mean(res) < 5:
                to_remove.append(tup.Index)
        

        df.drop(to_remove, inplace=True)
        return df

    def merge_if_three_names(df:pd.DataFrame):
        # TODO: !!!!
        pass             

    def clean_df(self, df):
        df = self.remove_unwanted_symbols(df)
        df = self.remove_unwanted_cells(df)

        df = self.remove_short_rows(df)
        # print('!!!',df)

        return df



In [4]:



class DocxParser:

    def get_docx_tables(self, filename, tab_id=None, **kwargs) -> list[pd.DataFrame]:
        """
            filename:   file name of a Word Document
            tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                        When [None] - return a list of DataFrames (parse all tables)
        """
        def read_docx_tab(tab, **kwargs):
            vf = io.StringIO()
            writer = csv.writer(vf)
            for row in tab.rows:
                writer.writerow(cell.text for cell in row.cells)
            vf.seek(0)
            return pd.read_csv(vf, **kwargs)

        doc = Document(filename)
        if tab_id is None:
            return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
        else:
            try:
                return read_docx_tab(doc.tables[tab_id], **kwargs)
            except IndexError:
                print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
                raise
            

    def convert_docx_to_df(self, filename: str) -> pd.DataFrame:
        assert filename.endswith('docx'), 'Формат должен быть .docx!'
            
        doc = Document(filename)
        # TODO: тут взять текст, который потом прикрутить к

        doc_tables = self.get_docx_tables(filename) 
        
        return doc_tables

        

In [7]:
class Parser:

    def __init__(self):
        self.cols_we_need = ['name','salary', 'position', 'department']
        self.all_docs: list[str]
        self.docx_parser = DocxParser()
        self.pdf_parser = PdfParser()
        self.parse_correct_headers = CorrectHeadersParser()
        self.incorrect_headers_parser = IncorrectHeaders()
        self.data_cleaner = DataCleaner()

    @staticmethod
    def rename_col(col: str) -> str:

        print('col before rename cols --', col)
        col = str(col).lower()
        if re.search(pattern='(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество)', string=col):
            return "name"

        elif re.search(pattern='(рублей|руб|cреднемесячная|зарпл.|плат[ы, а]|заработн[ой, ая] плат[а, ы]|cреднемесячн[ая, ой]|зарплат[а, ной, ы])', string=col):
            return "salary"

        elif re.search(pattern='(должност[ь, и, ей])', string=col): 

            return 'position'

        elif re.search(pattern='(предприяти[е,я]|учреждени[е,я]|юридическое лицо)', string=col):
            return 'department'

        return col


    @staticmethod
    def check_if_columns_ok(cols: tuple) -> bool:
        '''проверяем, есть ли в заголовках таблицы название предприятия и другая инфа'''
        
        cols = list(map(str, cols))
        cols = list(map(str.lower, cols))
        ok_cols = 0
        company_found = False
        for col in cols:
            company_pattern = '(предприяти[е,я]|учреждени[е,я]|юридическ[ое,ие])'
            res = re.search(pattern=company_pattern, string=col)            
            if res:
                company_found = True
                continue
                      
            name_salary_position_pattern = '(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество|плат[ы, а]|заработная|плата|cреднемесячн[ая, ой]|зарплат[а, ной, ы]|должность)'
            res = re.search(pattern=name_salary_position_pattern, string=col)
            if res:
                ok_cols+=1

        if company_found and ok_cols > 1:
            return True
        return False

    def parse_folder(self, file_path, destination_path):
        for file in os.listdir(file_path):
            try:
                df = self.parse_file(file_path)
                df.to_excel(destination_path + file_path)
            except Exception as ex:
                print(file_path)
                print(ex)
                print('===')


    def parse_file(self, file: str):
        
        if str(file).endswith('.pdf'):
            tables = self.pdf_parser.convert_pdf_to_df(file)
        
        elif file.endswith('docx'):
            tables = self.docx_parser.convert_docx_to_df(file)

        else:
            logger.error('Допустимы расширения: pdf, docx')

        parsed_tables = []

        for table in tables:
            # удалить кал из маленьких ячеек?
                         
            columns_ok = self.check_if_columns_ok(table)

            if not columns_ok:
                # колонки непправильные. идем пытаться найти нормальные. для этого мы весь файл передаем в Incorrect и скипаем цикл 
                
                parsing_ok, tables = self.incorrect_headers_parser.parse(Path(file))
                
                if parsing_ok:
                    for table in tables:
                        # оставляем нужные колонки
                        table.columns = [self.rename_col(col) for col in table.columns]
                        cols_to_leave = [col for col in table.columns if col in self.cols_we_need]
                        cols_to_leave = set(cols_to_leave)
                        table = table[cols_to_leave]
                        # проверяем на наличие вложенных таблиц и фио, разнесенных на несколько стаоблцов
                        table = self.parse_correct_headers.parse(table)
                        # убираем лишние ячейки и символы
                        table = self.data_cleaner.clean_df(table)
                        parsed_tables.append(table)
                    break

                else:                    
                    # TODO: сохранить файл в папку нераспаршенных 
                    logger.warning('Не удалось распарсить ----', file)

            elif columns_ok:                
                # если заголовки ок 
                # оставляем только нужные колонки 
                pass                
                table.columns = [self.rename_col(col) for col in table.columns]
                cols_to_leave = [col for col in table.columns if col in self.cols_we_need]
                cols_to_leave = set(cols_to_leave)
                table = table[cols_to_leave]
                # проверяем на наличие вложенных таблиц и фио, разнесенных на несколько стаоблцов
                table = self.parse_correct_headers.parse(table)
                # убираем лишние ячейки и символы
                table = self.data_cleaner.clean_df(table)
                parsed_tables.append(table)


        if isinstance(parsed_tables, list):
            if parsed_tables:
                concat_tables = pd.concat(parsed_tables)
                return concat_tables
    
        elif isinstance(parsed_tables, pd.DataFrame):
            if not parsed_tables.empty:
                return concat_tables
        


base = 'data_ids/pdf/converted/'
file = '189273_2020_Rektor,_prorektory,_glavnyi_bukhgalter.pdf'
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\100185_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(sport).pdf"

parser = Parser()
folder = Path(r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\для_которых_нужен_перевод_в_ворд')



# for e in os.listdir(folder):
#     if e.endswith('.pdf'):
#         res = parser.parse_file(folder / e)
#         with open(e+'.pkl','wb') as f:
#             pickle.dump(res, f)
            


NameError: name 'IncorrectHeaders' is not defined

In [ ]:

file = r'88767_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf'
file = r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\parser\88766_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pkl'

# with open('test.pkl', 'wb') as f:
#     pickle.dump(df, f)

with open(file, 'rb') as f:
    loaded_df = pickle.load(f)

# loaded_df
for e in os.listdir(folder):
    print(e)







88766_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf
88767_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf
88768_2017_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf
89001_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf
89058_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf
89060_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx
89060_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf
89062_2017_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf
89303_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii_(2017-2019_gody).pdf
89868_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf
89869_2018_Rukovoditeli,_zamestiteli_i_gla

In [23]:

 
# extract docx content
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\docx\102585_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(kul'tura).docx"
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\docx\101232_2018_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(obrazovanie).docx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\docx\184287_2018_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(obrazovanie).docx"
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\docx\102585_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(kul'tura).docx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\docx\179512_2020_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx"





class IncorrectHeaders:
    """класс для таблиц с неопределенными заголовками.
        1. Пытаемся найти название учреждений в объединенных ячейках.
        2. Если не получается, для учреждения берем текст, предшествующий таблице. 
    """

    # если прошелся по таблице и нашел вложения внутри - пусть это будет офис.
    # если не нашел - берем название офиса из абзацев вокруг таблиц (если их число плюс минус совпадает)

    def __init__(self):
        pass
        # TODO: добавить обработку чисто docx 

        # self.docx_parser = DocxParser()
        # self.pdf_parser = PdfParser()

    def drop_col_with_N(df:pd.DataFrame):
        expr = '(№|п/п)'
        for c in df.columns:
            if re.search(expr, c):
                df.drop(columns=c, inplace=True)

        return df

    def drop_short_cols(df: pd.DataFrame):
        len_df = df.applymap(len)
        for c in df.columns:
            if len_df[c].mean() < 4:
                df.drop(columns=c, inplace=True)
        return df

    
    def table_splitter(self, table: pd.DataFrame) -> tuple[bool, list[pd.DataFrame]]:
        '''разделяет таблицы, в которых учреждение указано внутри таблицы'''

        def check_if_same(my_array: list) -> bool:
            '''проверяем одинаковые ли колонки'''

            if len(set(my_array))>1:
                return False
            return True


        def get_indexes_to_split(table):
            '''определяем индекс строки таблицы, по которому надо разделить'''

            index_to_split = []
            for e in range(len(table)):
                cols = table.iloc[e,:].values
                if check_if_same(cols):
                    index_to_split.append(e)
            return index_to_split

                                                                                                #если разделили и нашли офис - True 
        def split_table(table: pd.DataFrame, index_to_split:Union[int, list[int]]) -> tuple[bool, list[pd.DataFrame]]:
            """разделяет таблицу в случае когда название учреждения поместили в середину вот так:
                -должность-  -имя-  -зарплата-
                        -ГБОУ школа 112-
                 директор     Ваня    100 руб

             """
            dfs = np.array_split(table, index_to_split)
            dfs = [e for e in dfs if len(e) > 0]

            result_dfs = []
            for df in dfs:
                office = df.iloc[0,:][0]
                df = df.iloc[1:,:] 
                df['office'] = office
                result_dfs.append(df)
            
            result_dfs = [e for e in result_dfs if not e.empty]
            try:
                result_dfs = pd.concat(result_dfs)
                return result_dfs
            except Exception as ex:
                print(ex)
                # print('rogue file---', table)
                
        index_to_split = get_indexes_to_split(table)

        if not index_to_split:
            return False, table

        splitted_dfs = split_table(table, index_to_split)
        return True, splitted_dfs

#        бесконечно распознается, не доходит до создания дока! 

    def convert_pdf_to_docx_to_find_info(self, filename: Path) -> Path:
        # переводим пфд в ворд
        assert str(filename).endswith('.pdf'), 'Файл должен быть в PDF !'
        folder = filename.parents[0]

        orig_file_name = filename.name.strip('.pdf')                 
        new_name = 'temp_to_delete_' + orig_file_name + '.docx'

        pdf2docx.parse(str(filename), str(folder / new_name))
        return folder / new_name

    @staticmethod
    def check_if_columns_ok(cols: tuple) -> bool:
        '''проверяем, есть ли в заголовках таблицы название предприятия и другая инфа'''
        
        cols = list(map(str, cols))
        cols = list(map(str.lower, cols))

        ok_cols = 0
        for col in cols:
            name_salary_position_pattern = '(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество|плат[ы, а]|заработная|плата|cреднемесячн[ая, ой]|зарплат[а, ной, ы]|должность)'
            
            res = re.search(pattern=name_salary_position_pattern, string=col)
            if res:
                ok_cols+=1

        if ok_cols > 1:
            return True

        return False


    def find_ok_cols(self, df:pd.DataFrame) -> pd.DataFrame:

        cols = df.columns
      
       # если колонки норм, отдаем df
        if self.check_if_columns_ok(cols):
            return df

        i = -1
        for _, row in df.iterrows():
            i+=1
            found_cols =  self.check_if_columns_ok(list(row))   
            if found_cols:

                df.columns = df.iloc[i,:]
                return df.iloc[i+1:,:]
            if i > 5:
                break
        
        # если колонки не ок -

        print('-'*10)
        print('не найдены нормальные колонки')
        print(df.head(3))
        print('-'*10)


#        raise ValueError('Incorrect Headers find_ok_cols --- в df не найдены нормальные колонки')


    
    def convert_pdf_to_dfs(self, filename: str) -> list[pd.DataFrame]:
#        print('filename in convert_pdf_to_df -----',filename)
        try:
            tables = camelot.read_pdf(str(filename), line_tol=2, joint_tol=10, line_scale=40, copy_text=[
                                    'v'], pages='1-end')  # , flavor='stream' row_tol=10
            tables = [e.df for e in tables]
            return tables

        except Exception as ex:
            logger.error('file --- %s', filename)
            logger.error('Exception --- %s', ex)


    def detect_headers_in_raw_doc(self, filename) -> list[pd.DataFrame]:
        
        def get_headers(filename: str) -> list[str]: #filename:docx

            doc = docx2python(filename)

            table_pattern = '(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество|должность)'

            offices = []
            gathering_office_info = ''

            for paragraph in doc.body_runs: #параграфы в виде вложенных листов

                paragraph = sum(sum(paragraph, []), [])
                paragraph_text = ''
                for e in paragraph:
                    try:
                        paragraph_text += ' ' + e[0] + ' '
                    except IndexError:
                        pass
                
                paragraph_text = paragraph_text.lower()
                its_table = re.findall(pattern=table_pattern, string=paragraph_text)
                
                if not its_table:
                    gathering_office_info+=paragraph_text
                
                elif its_table:
                    offices.append(gathering_office_info)
                    gathering_office_info=''

            return [e for e in offices if e]

        def compile_office_info_and_df(filename, departments: list) -> list[pd.DataFrame]:
            # все правильно. логика такая, что камелотом лучше парсить!
            # а док только для загов таблиц

            tables = self.convert_pdf_to_df_and_find_tables(filename)

            ok_dfs = []

            print('Количество заголовков --- ', len(departments))
            print('Количество таблиц --- ', len(tables))
            
            if len(departments) - len(tables) == 1:
                departments.pop()


            if len(departments) == len(tables):
                for table, dep in zip(tables, departments):
                    table['Учреждение'] = dep
                    table['Учреждение'][0] = 'Учреждение'
 
                    ok_dfs.append(table)
          
                return ok_dfs
            
            raise ValueError('Разное число таблиц и учреждений')
        
        temp_docfile = self.convert_pdf_to_docx_to_find_info(filename) # получили path временного docx файла
        departments = get_headers(temp_docfile)
        dfs = compile_office_info_and_df(filename, departments)
        return dfs


    def parse(self, filename: Path) -> tuple[bool, pd.DataFrame]:
        # пытаемся найти учреждения в теле таблиц

        #TODO: добавить проверку doc или pdf

        # должны быть просто таблицы
        # и вся обработка должна быть тут, по этапам. иначе макароны
        tables = self.convert_pdf_to_df_and_find_tables(filename)
        # дропаем маленькие колонки
        tables = [self.drop_col_with_N(e) for e in tables]
        tables = [self.drop_short_cols(e) for e in tables]
        tables = [self.find_ok_cols(e) for e in tables]
        

        # tables = [e for e in tables if type(e) == pd.DataFrame]
        


        # tables_with_ok_headers = []
        # if not tables:
        #     return False, []
        
        # for table in tables:
        #     res, df = self.table_splitter(table)
        #     if res:
        #         tables_with_ok_headers.append(df)

        #     if not res:
        #         del tables
        #         # идем парсить весь док, чтобы достать учреждения из текста перед таблицей
        #         dfs = self.detect_headers_in_raw_doc(filename)
        #         for df in dfs:
        #             tables_with_ok_headers.append(df)
        #         break
        

        
        #TODO: переделать удаление временного дока

        return bool(tables_with_ok_headers), tables_with_ok_headers        
        

       # 'temp_to_delete_D:\\PROGR\\LEARN_PYTHON\\Declarator\\declarations-parser\\data_ids\\pdf\\converted\\для которых нужен перевод в ворд\\88766_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx'
        
    


file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\для_которых_нужен_перевод_в_ворд\89001_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\100185_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(sport).pdf"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\104563_2019_Rektor,_prorektory,_glavnyi_bukhgalter.pdf"

incor_parser = IncorrectHeaders()

temp_file = incor_parser.convert_pdf_to_docx_to_find_info(Path(file))
temp_file
# print(len(res))
# for df in res:
#     print(df.iloc[0,1])



[INFO] Start to convert D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\104563_2019_Rektor,_prorektory,_glavnyi_bukhgalter.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[INFO] [3/4] Parsing pages...
[INFO] (1/1) Page 1
[INFO] [4/4] Creating pages...
[ERROR] [WinError 32] Процесс не может получить доступ к файлу, так как этот файл занят другим процессом: 'D:\\PROGR\\LEARN_PYTHON\\Declarator\\declarations-parser\\data_ids\\pdf\\converted\\temp_to_delete_104563_2019_Rektor,_prorektory,_glavnyi_bukhgalter.docx'


WindowsPath('D:/PROGR/LEARN_PYTHON/Declarator/declarations-parser/data_ids/pdf/converted/temp_to_delete_104563_2019_Rektor,_prorektory,_glavnyi_bukhgalter.docx')

In [72]:
def get_headers(filename: str) -> list[str]: #filename:docx

        doc = docx2python(filename)

        table_pattern = '(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество|должность)'

        offices = []
        gathering_office_info = ''

        for paragraph in doc.body_runs: #параграфы в виде вложенных листов

            paragraph = sum(sum(paragraph, []), [])
            paragraph_text = ''
            for e in paragraph:
                try:
                    paragraph_text += ' ' + e[0] + ' '
                except IndexError:
                    pass
            
            paragraph_text = paragraph_text.lower()
            its_table = re.findall(pattern=table_pattern, string=paragraph_text)
            
            if not its_table:
                gathering_office_info+=paragraph_text
            
            elif its_table:
                offices.append(gathering_office_info)
                gathering_office_info=''

        return [e for e in offices if e]

[' \t ']

In [86]:

def get_headers(doc_file):

    doc = docx2python(doc_file)

    table_pattern = '(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество|должность)'

    offices = []
    gathering_office_info = ''

    for paragraph in doc.body_runs: #параграфы в виде вложенных листов

        paragraph = sum(sum(paragraph, []), [])
        paragraph_text = ''
        for e in paragraph:
            try:
                paragraph_text += ' ' + e[0] + ' '
            except IndexError:
                pass
        
        paragraph_text = paragraph_text.lower()
        its_table = re.findall(pattern=table_pattern, string=paragraph_text)
        print(paragraph_text)
        print('===')
        if not its_table:
            gathering_office_info+=paragraph_text
        
        elif its_table:
            offices.append(gathering_office_info)
            gathering_office_info=''

    return offices

# doc_file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\для_которых_нужен_перевод_в_ворд\temp_to_delete_89001_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx"
doc_file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\100185_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(sport).docx"
doc_file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\104563_2019_Rektor,_prorektory,_glavnyi_bukhgalter.docx"

doc = docx2python(temp_file)
for paragraph in doc.body_runs: #параграфы в виде вложенных листов
    paragraph_text = ''

    paragraph = sum(sum(paragraph, []), [])
    for e in paragraph:
        try:
            paragraph_text += ' ' + e[0] + ' '
        except IndexError:
            print('Ошибка!')
            

    print(paragraph_text)
    print('===')

# paragraph_text

# offices = get_headers(temp_file)
# offices

# len([e for e in offices if e])


Ошибка!
 	 
===
 № 
п/п   Наименование должности   ФИО   Среднемесячная 
заработная плата, руб.   1   Ректор   Федорук Михаил Петрович   495 048,0   2   Первый проректор - проректор по финансово-экономической и 
инновационной деятельности   Голушко Сергей Кузьмич   324 989,0   3   Проректор по учебной работе   Саблина Светлана Геннадьевна   295 871,0   4   Проректор по научно-исследовательской деятельности   Чуркин Дмитрий Владимирович   378 002,0   5   Проректор по программам развития   Окунев Алексей Григорьевич   311 220,0   6   Проректор по общим вопросам   Малиновский Сергей Игоревич   307 303,0   7   Главный бухгалтер    Тарских Надежда Алексеевна   317 370,0  
===
Ошибка!

===


In [19]:
temp_doc = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\temp_to_delete_104563_2019_Rektor,_prorektory,_glavnyi_bukhgalter.docx"

doc = docx2python(temp_doc)

table_pattern = '(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество|должность)'

for paragraph in doc.body_runs: #параграфы в виде вложенных листов
    paragraph = sum(sum(paragraph, []), [])
    paragraph = [e for e in paragraph if e]
    paragraph_text = ''
    for e in paragraph:
        print(e)
        try:
            paragraph_text += ' ' + e[0] + ' '
        except IndexError:
            print('ошибка!')

    print(paragraph_text)
    print('==')


['\t', '\t', '\t', '\t', 'ИНФОРМАЦИЯ \nо  среднемесячной заработной плате ректора, проректоров и главного бухгалтера федерального государственного автономного образовательного учреждения ', '\t', 'высшего образования «Новосибирский национальный исследовательский государственный университет» \n', '\t', ' за период с 1 января 2019г. по 31 декабря 2019г. ']
 	 
==
['№ \nп/п ']
['Наименование должности ']
['ФИО ']
['Среднемесячная \nзаработная плата, руб. ']
['1 ']
['Ректор ']
['Федорук Михаил Петрович ']
['495 048,0 ']
['2 ']
['Первый проректор - проректор по финансово-экономической и \nинновационной деятельности ']
['Голушко Сергей Кузьмич ']
['324 989,0 ']
['3 ']
['Проректор по учебной работе ']
['Саблина Светлана Геннадьевна ']
['295 871,0 ']
['4 ']
['Проректор по научно-исследовательской деятельности ']
['Чуркин Дмитрий Владимирович ']
['378 002,0 ']
['5 ']
['Проректор по программам развития ']
['Окунев Алексей Григорьевич ']
['311 220,0 ']
['6 ']
['Проректор по общим вопросам ']
['Ма

In [28]:
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\104649_2019_Rektor,_prorektory,_glavnyi_bukhgalter.pdf"
incor_parser = IncorrectHeaders()
pdf_parser = PdfParser()
df = pdf_parser.convert_pdf_to_df(file)




[INFO] xref found: pos=b'99570'
[INFO] read_xref_from: start=99570, token=/b'xref'
[INFO] xref objects: {1: (None, 9, 0), 2: (None, 68, 0), 3: (None, 108, 0), 4: (None, 444, 0), 5: (None, 493, 0), 6: (None, 1180, 0), 7: (None, 2037, 0), 8: (None, 2782, 0), 9: (None, 3535, 0), 10: (None, 4330, 0), 11: (None, 5097, 0), 12: (None, 5879, 0), 13: (None, 6611, 0), 14: (None, 6754, 0), 15: (None, 6942, 0), 16: (None, 7182, 0), 17: (None, 7847, 0), 18: (None, 9710, 0), 19: (None, 9810, 0), 20: (None, 9908, 0), 21: (None, 10009, 0), 22: (None, 10115, 0)}
[INFO] trailer: {'Size': 23, 'Root': <PDFObjRef:4>, 'Info': <PDFObjRef:2>}
[INFO] trailer: {'Size': 23, 'Root': <PDFObjRef:4>, 'Info': <PDFObjRef:2>}
[INFO] Pages: Kids=[<PDFObjRef:3>]
[INFO] Page: {'Contents': [<PDFObjRef:5>, <PDFObjRef:6>, <PDFObjRef:7>, <PDFObjRef:8>, <PDFObjRef:9>, <PDFObjRef:10>, <PDFObjRef:11>, <PDFObjRef:12>], 'CropBox': [0, 0, 612.48, 842.4], 'MediaBox': [0, 0, 612.48, 842.4], 'Parent': <PDFObjRef:1>, 'Resources': {'Fon

In [77]:
df = dfs[0]



'''
2. придумать, как объединять таблицы на разных страницах.

'''

def drop_col_with_N(df:pd.DataFrame):
    expr = '(№|п/п)'
    for c in df.columns:
        if re.search(expr, c):
            df.drop(columns=c, inplace=True)

    return df

def drop_short_cols(df: pd.DataFrame):
    len_df = df.applymap(len)
    for c in df.columns:
        if len_df[c].mean() < 4:
            df.drop(columns=c, inplace=True)
    return df




In [159]:
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\converted\177695_2020_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.pdf"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\pdf\101640_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(kul'tura).pdf"

# dfs = incor_parser.convert_pdf_to_df_and_find_tables(file)
# dfs[0]
#dfs = pdf_parser.convert_pdf_to_df(file)

#
"""
есть список таблиц. 
если у первой таблицы найдены заголовки
а у других таблиц нет - присовокупляем все к первой

"""
list_of_dfs = []
my_hash = {
    'ok_df':True, 'ne_ok_df':False
}
correct = False
df_to_concat = ''


# если на этом этапе колонки не найдены. то таблицу - в топку

all_oks = my_hash.values()
if all(all_oks) or not all(all_oks):
    print('дропаем')


for df, ok_cols in my_hash.items():
    if ok_cols:
        df_to_concat = df
    elif not ok_cols and df_to_concat:
        print('КОНКАТИНИРУЕМ')


# ok_df = incor_parser.find_ok_cols(dfs[0])
# ne_ok = dfs[1]
# ne_ok.columns = ok_df.columns
# pd.concat([ok_df,ne_ok])




,№ \nп/п,ФИО,Должность,Среднемесячная заработная \nплата за 2019 год (руб.)
0,№ \nп/п,ФИО,Должность,Среднемесячная заработная \nплата за 2019 год ...
1,1.,Старинцева Оксана \nГульвариевна,Директор муниципального \nучреждения «Районный...,"74357,52"
2,2.,Шахназарян Татьяна \nМихайловна,Директор муниципального \nучреждения культуры ...,"44323,2"
3,3.,Сачко Ирина Юрьевна,Директор муниципального \nучреждения культуры ...,"54180,36"
4,4.,Лисаченко Зоя \nВладимировна,Директор бюджетного \nучреждения дополнительно...,"45651,44"
5,5.,Семерижная Любовь \nИвановна,Директор муниципального \nучреждения «Районный...,"51046,40"
6,6.,Коваленко Татьяна \nПавловна,Директор муниципального \nучреждения «Районный...,"41884,03"
7,7.,Ладыко Лидия Николаевна,главный бухгалтер \nмуниципального учреждения ...,"49976,55"
8,8.,Миронова Елена \nВалерьевна,главный бухгалтер \nмуниципального учреждения ...,"35555,00"
9,9.,Бондаренко Марина \nВикторовна,Г лавный бухгалтер бюджетного \nучреждения доп...,"40507,35"


In [162]:
l = [False, False]

not all(l)

True

,index,0,1,2,3
0,0,,,образования «Детская школа \nискусств»,
1,1,10.,Смирнова Светлана \nАнатольевна,главный бухгалтер \nмуниципального учреждения ...,"32902,72"
2,2,11.,Чебан Сергей Иванович,заместитель директора по \nхозяйственной части...,"52862,41"
3,3,12.,Лозина Надежда \nАнатольевна,Заместитель директора по \nучебной части муниц...,"31832,05"
4,4,13.,Котченко Виктор \nВладимирович,Заместитель директора по АХЧ \nмуниципального ...,"35329,22"
